### Dask + XArray
Xarray can automatically wrap its data in dask arrays. This capability turns xarray into an extremely powerful tool for Big Data earth science. To see this in action, we will download a fairly large dataset to analyze. This file contains 1 year of daily data from the AVISO sea-surface height satellite altimetry dataset.

In [1]:
import xarray as xr
import glob
import bottleneck

In [17]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=1) # Change n_workers = 
client

Client Scheduler: tcp://127.0.0.1:40475 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 1.08 TB


Let's load the first file as a regular xarray dataset.

In [3]:
ds_first = xr.open_dataset('/silod5/boergel/model_exp/MOM_9nm_hiresaff/v01r03/balt-8nm-brine-v01-r03_1850/ocean_day3d.nc')
ds_first

<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 101, st_ocean: 100, sw_edges_ocean: 101, sw_ocean: 100, time: 11, xt_ocean: 91, xu_ocean: 91, yt_ocean: 102, yu_ocean: 102)
Coordinates:
  * xt_ocean        (xt_ocean) float64 8.12 8.36 8.6 8.84 ... 29.24 29.48 29.72
  * yt_ocean        (yt_ocean) float64 53.64 53.76 53.88 ... 65.52 65.64 65.76
  * time            (time) object 1850-02-15 00:00:00 ... 1850-12-16 12:00:00
  * nv              (nv) float64 1.0 2.0
  * xu_ocean        (xu_ocean) float64 8.24 8.48 8.72 8.96 ... 29.36 29.6 29.84
  * yu_ocean        (yu_ocean) float64 53.7 53.82 53.94 ... 65.58 65.7 65.82
  * st_ocean        (st_ocean) float64 0.25 0.7572 1.315 ... 222.5 225.5 228.5
  * st_edges_ocean  (st_edges_ocean) float64 -0.001806 0.5018 ... 227.0 230.0
  * sw_ocean        (sw_ocean) float64 0.5036 1.036 1.624 ... 224.0 227.0 230.0
  * sw_edges_ocean  (sw_edges_ocean) float64 0.25 0.7572 1.315 ... 228.5 230.0
    geolon_t        (yt_ocean, xt_ocean) float32 ..

In [4]:
ds_first.nbytes / 1e6

245.348672

This one file is about 245 MB. So 50 of them will be nearly 12,250 GB. 

In [5]:
files  = glob.glob("/silod5/boergel/model_exp/MOM_9nm_hiresaff/v01r03/balt-8nm-brine-v01-r03_18*/ocean_day3d.nc")
ds = xr.open_mfdataset(files, concat_dim="time")
ds

<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 101, st_ocean: 100, sw_edges_ocean: 101, sw_ocean: 100, time: 599, xt_ocean: 91, xu_ocean: 91, yt_ocean: 102, yu_ocean: 102)
Coordinates:
  * xt_ocean        (xt_ocean) float64 8.12 8.36 8.6 8.84 ... 29.24 29.48 29.72
  * yt_ocean        (yt_ocean) float64 53.64 53.76 53.88 ... 65.52 65.64 65.76
  * nv              (nv) float64 1.0 2.0
  * xu_ocean        (xu_ocean) float64 8.24 8.48 8.72 8.96 ... 29.36 29.6 29.84
  * yu_ocean        (yu_ocean) float64 53.7 53.82 53.94 ... 65.58 65.7 65.82
  * st_ocean        (st_ocean) float64 0.25 0.7572 1.315 ... 222.5 225.5 228.5
  * st_edges_ocean  (st_edges_ocean) float64 -0.001806 0.5018 ... 227.0 230.0
  * sw_ocean        (sw_ocean) float64 0.5036 1.036 1.624 ... 224.0 227.0 230.0
  * sw_edges_ocean  (sw_edges_ocean) float64 0.25 0.7572 1.315 ... 228.5 230.0
    geolon_t        (yt_ocean, xt_ocean) float32 8.12 8.36 8.6 ... nan nan nan
    geolat_t        (yt_ocean, xt_ocean) float32 

Note that the values are not displayed, since that would trigger computation.

In [6]:
salt = ds.salt
salt

<xarray.DataArray 'salt' (time: 599, st_ocean: 100, yt_ocean: 102, xt_ocean: 91)>
dask.array<shape=(599, 100, 102, 91), dtype=float32, chunksize=(11, 100, 102, 91)>
Coordinates:
  * xt_ocean  (xt_ocean) float64 8.12 8.36 8.6 8.84 ... 29.0 29.24 29.48 29.72
  * yt_ocean  (yt_ocean) float64 53.64 53.76 53.88 54.0 ... 65.52 65.64 65.76
  * st_ocean  (st_ocean) float64 0.25 0.7572 1.315 1.934 ... 222.5 225.5 228.5
    geolon_t  (yt_ocean, xt_ocean) float32 8.12 8.36 8.6 8.84 ... nan nan nan
    geolat_t  (yt_ocean, xt_ocean) float32 53.64 53.64 53.64 ... nan nan nan
  * time      (time) object 1850-02-15 00:00:00 ... 1899-12-16 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_salinity

In [7]:
salt.isel(time=1, st_ocean=1).plot()

In [18]:
salt_mean = salt.mean()
%time salt_mean.load()

CPU times: user 527 ms, sys: 29.4 ms, total: 556 ms
Wall time: 12.9 s


<xarray.DataArray 'salt' ()>
array(15.656407, dtype=float32)

In [16]:
salt_mean

<xarray.DataArray 'salt' ()>
array(15.656407, dtype=float32)

Accessing the underlying array of data is done via the data property. Here we can see that we have a Dask array. If this array were to be backed by a NumPy array, this property would point to the actual values in the array.

In [1]:
client.close()

NameError: name 'client' is not defined